In [1]:
import numpy as np
from pandas_plink import read_plink
from limix_lmm import LMM
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from statsmodels.stats.multitest import multipletests
import seaborn as sns

In [2]:
import scipy
import scipy.stats as st

if not hasattr(scipy, 'dot'):
    scipy.dot = np.dot
if not hasattr(scipy, 'einsum'):
    scipy.einsum = np.einsum
if not hasattr(scipy, 'log'):
    scipy.log = np.log
if not hasattr(scipy, 'sign'):
    scipy.sign = np.sign
if not hasattr(scipy, 'sqrt'):
    scipy.sqrt = np.sqrt

In [3]:
def simulate_pheno(X, idx_caus, var_expl, rng, direction=None):
    # Ensure that the number of causal variant indices matches the number of variances explained.
    assert len(idx_caus) == len(var_expl)

    # If no direction is provided, randomly assign a positive or negative direction for each causal variant.
    if direction is None:
        direction = 2. * (rng.random(len(idx_caus)) > 0.5) - 1.
    # Ensure that the number of directions matches the number of causal variant indices.
    assert len(idx_caus) == len(direction)

    # Compute the remaining variance after accounting for the variance explained by the causal variants.
    ve = 1 - var_expl.sum()
    # Ensure that the total variance explained by causal variants is less than 1.
    assert ve > 0, 'sum(var_expl) should be < 1'

    # Compute the effect sizes for the causal variants based on the variance they explain and their direction.
    beta = np.sqrt(var_expl) * direction

    # Extract the columns of X corresponding to the causal variants and standardize them.
    Xc = X[:, idx_caus]
    Xc = (Xc - Xc.mean(0)) / Xc.std(0)

    # Compute the genetic component of the phenotype.
    yg = Xc.dot(beta)[:, None]
    # Compute the noise component of the phenotype.
    yn = np.sqrt(ve) * rng.standard_normal((X.shape[0], 1))

    # Sum the genetic and noise components to get the simulated phenotype.
    y = yg + yn

    # Initialize the real effect sizes for all variants in X as zeros.
    beta_real = np.zeros(X.shape[1])
    # Update the real effect sizes for the causal variants.
    beta_real[idx_caus] = beta

    # Standardize the phenotypic values to have mean 0 and standard deviation 1.
    ystd = y.std()
    y = (y - y.mean()) / ystd
    # Adjust the real effect sizes accordingly after standardizing y.
    beta_real = beta_real / ystd

    return y, beta_real

In [4]:
def qq_plot(p_values, title):
    """
    Create a QQ plot given a list of p-values.

    Parameters:
    - p_values: list of p-values
    - title: title for the plot
    """

    # Sort p-values
    observed = -np.log10(np.sort(p_values))
    expected = -np.log10(np.arange(1, len(p_values) + 1) / (len(p_values) + 2))

    # Create the QQ plot
    plt.scatter(expected, observed, marker='.')
    plt.plot([0, max(expected)], [0, max(expected)], color='red', linestyle='--')
    plt.xlabel('Expected -log10(P-value)')
    plt.ylabel('Observed -log10(P-value)')
    plt.title(title)

In [5]:
bfile = "/Users/oykusuoglu/gobi/gobi_gwas/universal_data/preprocessing/chr22_preprocessed"

In [6]:
bim,fam, G = read_plink(bfile)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 47.21it/s]
/opt/anaconda3/envs/gobi_venv/lib/python3.10/site-packages/pandera/_pandas_deprecated.py:146: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [7]:
n_snps = bim.shape[0]
n_samples = fam.shape[0]
n_snps, n_samples

(59743, 2504)

In [8]:
X_real = G.compute().T

In [9]:
mu_full = X_real.mean(axis=0)
sd_full = X_real.std(axis=0, ddof=0)
keep_full = sd_full > 1e-12
keep_idx  = np.where(keep_full)[0] 
standardized_X = (X_real[:, keep_full] - mu_full[keep_full]) / sd_full[keep_full]

In [10]:
bim_kept = bim.iloc[keep_idx].copy().reset_index(drop=True)

bim_kept["orig_bim_idx"] = keep_idx

In [11]:
h2 = [0.6]
n_causal = [10]

In [12]:
r2_matrix = np.zeros((len(h2), len(n_causal)))
spearman_matrix = np.zeros((len(h2), len(n_causal)))
count_v = 0
count_h = 0

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
idx = np.arange(len(fam))

idx_train, idx_test = train_test_split(idx, test_size=0.2, random_state=42, shuffle=True)

train_ids = fam.loc[idx_train, ["fid","iid"]].copy()
test_ids  = fam.loc[idx_test,  ["fid","iid"]].copy()


train_ids.to_csv("/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/splits/train.keep",
                 sep="\t", index=False, header=False)
test_ids.to_csv("/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/splits/test.keep",
                sep="\t", index=False, header=False)
                

X_train = standardized_X[idx_train]
X_test  = standardized_X[idx_test]

In [15]:
from pathlib import Path
base_dir = Path("/Users/oykusuoglu/gobi/gobi_gwas/oyku/plots/plots_w_new_pcs")

In [16]:
for h in h2:
    for n in n_causal:
        rng = np.random.default_rng(42)
        idx_caus_loop = rng.choice(standardized_X.shape[1], size=n, replace=False)
        var_expl_loop = np.repeat(h/n, n)

        y_loop, beta_real_loop = simulate_pheno(standardized_X, idx_caus_loop, var_expl_loop, rng)

        pheno = fam[["fid","iid"]].copy()
        pheno.columns = ["FID","IID"]
        pheno["y"] = y_loop.reshape(-1)

        pc_path = "/Users/oykusuoglu/gobi/gobi_gwas/oyku/data/pca/before_qc/chr22_raw_pca10.eigenvec.zscore"
        pcs = pd.read_csv(pc_path, sep=r"\s+", header=0, engine="python")
        pcs.columns = ["FID","IID"] + [f"PC{i}" for i in range(1, pcs.shape[1]-1)]

        df = pheno.merge(pcs, on=["FID","IID"], how="inner", validate="one_to_one")

        k = 10
        F = np.column_stack([np.ones((df.shape[0], 1)),
                            df[[f"PC{i}" for i in range(1, k+1)]].to_numpy()])
        
        y_train = y_loop[idx_train]
        y_test  = y_loop[idx_test]
        F_train = F[idx_train]
        F_test  = F[idx_test]
        
        
        lmm = LMM(y_train, F_train)
        lmm.process(X_train)
        pv = lmm.getPv()
        beta = lmm.getBetaSNP()
        beta_ste = lmm.getBetaSNPste()

        outdir = base_dir / f"h2_{h}/causal_vars{n}"
        outdir.mkdir(parents=True, exist_ok=True)


        fig = plt.figure()
        qq_plot(pv, f'QQ Plot for heritability: {h} number of causal variants: {n}')
        fig.tight_layout()
        fig.savefig(outdir / f"qq_h2_{h}_causal_vars{n}.png", dpi=300, bbox_inches="tight")
        plt.close(fig)

        x = bim_kept['pos'].values
        fig2 = plt.figure(figsize=(10, 6))
        ax1 = fig2.add_subplot(2, 1, 1)
        ax2 = fig2.add_subplot(2, 1, 2, sharex=ax1)

        ax1.set_title(f"Real effect size for heritability: {h} number of causal variants: {n}")
        ax1.plot(x, beta_real_loop, ".k")
        ax1.set_ylabel("eff size")

        ax2.set_title(f"GWAS results for heritability: {h} number of causal variants: {n}")
        ax2.plot(x, -np.log10(pv), ".k")
        ax2.set_ylabel(r"-log$_{10}$ P")
        ax2.set_xlabel("position")

        fig2.tight_layout()
        fig2.savefig(outdir / f"effects_gwas_h2{h}_causal_vars{n}.png",
                    dpi=300, bbox_inches="tight")
        plt.close(fig2)

        
        

In [17]:
clumped = pd.read_csv("/Users/oykusuoglu/gobi/gobi_gwas/universal_data/clumping/h2_0.6/causal_10/h2_0.6_causal_10_clumped.clumped", sep=r'\s+', engine='python')

In [18]:
clumped

,CHR,F,SNP,BP,P,TOTAL,NSIG,S05,S01,S001,S0001,SP2
0,22,1,rs138948,43181530,1.440000e-40,33,1,1,0,2,29,"rs6519365(1),rs138941(1),rs138942(1),rs138943(..."
1,22,1,rs5753682,31538427,2.320000e-34,172,0,0,2,1,169,"rs73158585(1),rs5997959(1),rs9609263(1),rs7315..."
2,22,1,rs4823720,47878287,5.420000e-33,50,0,0,1,1,48,"rs4823716(1),rs4823717(1),rs62224236(1),rs1210..."
3,22,1,rs200742767,40569771,2.760000e-32,237,0,0,0,1,236,"rs6001894(1),rs9611325(1),rs6001898(1),rs55769..."
4,22,1,rs2015580,31930035,3.290000e-30,146,12,10,14,10,100,"rs76030575(1),rs5994426(1),rs138280(1),rs34733..."
5,22,1,rs2301555,23152717,3.000000e-29,196,6,0,10,28,152,"rs5996446(1),rs9612207(1),rs9612210(1),rs76215..."
6,22,1,rs73158578,31308466,4.930000e-29,35,0,0,1,7,27,"rs147255832(1),rs12485186(1),rs73156792(1),rs5..."
7,22,1,rs5748017,19163333,1.260000e-25,26,3,0,1,0,22,"rs2871016(1),rs1468269(1),rs2871018(1),rs28606..."
8,22,1,rs12159641,45031505,2.080000e-24,69,5,2,5,14,43,"rs5766277(1),rs8142752(1),rs8136316(1),rs81386..."
9,22,1,rs2158335,19101094,2.570000e-22,379,32,1,8,6,332,"rs5992351(1),rs2023635(1),rs2023636(1),rs17810..."


In [19]:
lead_snps = clumped["SNP"].astype(str).tolist()

In [20]:
len(lead_snps)

36

In [21]:
snp_to_col = pd.Series(index=bim_kept["snp"].astype(str), data=range(bim_kept.shape[0]))
cols = snp_to_col.reindex(lead_snps).dropna().astype(int).to_numpy()

In [22]:
X_clump_train = X_train[:, cols] 
X_clump_test = X_test[:, cols]   

In [23]:
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error

In [24]:
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

regr = linear_model.LinearRegression()
regr.fit(X_clump_train, y_train)

y_pred = regr.predict(X_clump_test)

In [25]:
from scipy import stats
spearman = stats.spearmanr(y_test, y_pred)
print("R2", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("spearman r: ", spearman.statistic)

R2 0.5698198514473514
MSE: 0.44389006893513877
spearman r:  0.7459687477634374
